In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
import seaborn as sns

In [2]:
import requests
from urllib.request import urlopen
import json

In [3]:
import spark
import pyspark
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import *


In [4]:
spark = SparkSession.builder.appName('capstone_project').getOrCreate()


In [ ]:

#read the database username and password from secret.txt
secrets_file = os.path.join("files", "secret.txt")
with open(secrets_file, "r") as f:		
    lines = f.readlines()
for line in lines:
    words = line.split("=")
    if (words[0] == "user"):
        user = words[1].strip()
    elif (words[0] == "password"):
        password = words[1].strip()


In [5]:
url = 'https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json'

In [6]:

try:
    response = requests.get(url,timeout=3)
    response.raise_for_status()
except requests.exceptions.RequestException as err:
    print ("Oops: Something Else",err)
except requests.exceptions.HTTPError as errh:
    print ("Http Error:",errh)
except requests.exceptions.ConnectionError as errc:
    print ("Error Connecting:",errc)
except requests.exceptions.Timeout as errt:
    print ("Timeout Error:",errt)     
#print(response.status_code)

200


In [17]:
#loan_data = json.loads(response.text)
loan_datafile = os.path.join("files", "loan_datafile.json")

loan_data = response.json()
#print(loan_data)
with open(loan_datafile,'w') as f:
    json.dump(loan_data,f)
#print(loan_data)

In [18]:
loan_dataset = spark.read.format('json').load(loan_datafile)


In [19]:
loan_dataset.printSchema()

root
 |-- Application_ID: string (nullable = true)
 |-- Application_Status: string (nullable = true)
 |-- Credit_History: long (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Income: string (nullable = true)
 |-- Married: string (nullable = true)
 |-- Property_Area: string (nullable = true)
 |-- Self_Employed: string (nullable = true)



In [20]:
loan_dataset.show()

+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|Application_ID|Application_Status|Credit_History|Dependents|   Education|Gender|Income|Married|Property_Area|Self_Employed|
+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|      LP001002|                 Y|             1|         0|    Graduate|  Male|medium|     No|        Urban|           No|
|      LP001003|                 N|             1|         1|    Graduate|  Male|medium|    Yes|        Rural|           No|
|      LP001005|                 Y|             1|         0|    Graduate|  Male|   low|    Yes|        Urban|          Yes|
|      LP001006|                 Y|             1|         0|Not Graduate|  Male|   low|    Yes|        Urban|           No|
|      LP001008|                 Y|             1|         0|    Graduate|  Male|medium|     No|        Urban|           No|


In [ ]:
f.close()

In [25]:
loan_dataset.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
.option("dbtable", "creditcard_capstone.cdw_sapp_loan_application") \
.option("user", user) \
.option("password", password) \
.save()

In [ ]:
spark.close()

In [ ]:
#loan_dataframe.head(5)